In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Thu May  4 15:37:09 2017

@author: zhouyi
"""
import glob
import os
# if you are using OPENBLAS, you might want to turn this option on. Otherwise, joblib might get stuck
# os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import scipy.sparse
import pandas as pd

import expomf_cov
from expomf_cov import get_mu,inv_logit


DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/ml-1m'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
    
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
def load_data(csv_file, shape=(n_users, n_songs)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['rating']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.csv'))
# binarize the data
train_data.data = np.ones_like(train_data.data)

print train_data.shape
print train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.csv'))
# binarize the data
vad_data.data = np.ones_like(vad_data.data)

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.csv'))
# binarize the data
test_data.data = np.ones_like(test_data.data)

pi = np.loadtxt(os.path.join(DATA_ROOT, 'genre_movies_vector'), dtype='float32',skiprows=1)

(4200, 2019)
(636096,)


In [4]:
# sanity check to make sure all the venues has its corresponding feature    
for i, s in enumerate(unique_sid):
    assert s == "%d" % pi[i, 0]
    #check if there is zero factor, it should be sumed to 1
    assert sum(pi[i, 1:]) > 0.9

pi = pi[:, 1:]             
noise=np.random.rand(2019,18)/40
pi+=noise

In [5]:
n_components = 18
max_iter = 15
n_jobs = 20
lam = 1e-5
# here we use the best performing init_mu from per-item \mu_i experiment
init_mu = 0.1
max_epoch = 10

save_dir="Movielens_params_K%d_lam%1.0E_initmu%1.0E_maxepoch%d" % (n_components, lam, init_mu, max_epoch)


coder = expomf_cov.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=1000, 
                          batch_sgd=10, max_epoch=max_epoch, init_std=0.01,
                          n_jobs=n_jobs, random_state=98765, save_params=True, save_dir=save_dir, 
                          early_stopping=True, verbose=True, 
                          lam_y=1., lam_theta=lam, lam_beta=lam, lam_nu=lam, init_mu=init_mu, learning_rate=.5)

coder.fit(train_data, pi, vad_data=vad_data,k=20)

ITERATION #0
	Updating user factors: time=2.83
	Updating item factors: time=2.25
	Updating user exposure factors...
		Epoch #0: initial validation loss = 1776.250
		Epoch #0: validation loss = 1670.484
		Epoch #1: initial validation loss = 1700.696
		Epoch #1: validation loss = 1702.813
		Epoch #2: initial validation loss = 1608.493
		Epoch #2: validation loss = 1603.740
		Epoch #3: initial validation loss = 1776.467
		Epoch #3: validation loss = 1775.429
		Epoch #4: initial validation loss = 2020.725
		Epoch #4: validation loss = 2012.521
		Epoch #5: initial validation loss = 1691.177
		Epoch #5: validation loss = 1695.436
		Epoch #6: initial validation loss = 1755.226
		Epoch #6: validation loss = 1759.432
		Epoch #7: initial validation loss = 1780.237
		Epoch #7: validation loss = 1780.844
		Epoch #8: initial validation loss = 1641.027
		Epoch #8: validation loss = 1641.652
		Epoch #9: initial validation loss = 1760.506
		Epoch #9: validation loss = 1761.887
	Updating user exposure 

ExpoMF(batch_sgd=10, batch_size=1000, early_stopping=True, init_std=0.01,
    max_epoch=10, max_iter=15, n_components=18, n_jobs=20,
    random_state=98765,
    save_dir='Movielens_params_K18_lam1E-05_initmu1E-01_maxepoch10',
    save_params=True, verbose=True)